# Naive Bayesian Classifier

In [2]:
import pandas as pd
import numpy as np
import math
import statistics
from collections import defaultdict, Counter

In [3]:
# Keep track of:
# 1) number of occurances of each word in a class ("all_words" dictionary)
# 2) the total number of words per class ("total_frequency_by_class" list)
# 3) quantities of each class ("numbers_of_classes" list)


def empty_class_list():
    return [0, 0, 0, 0]


def track_frequencies(data, num_examples, class_indexes, b=None):

    numbers_of_classes = [0, 0, 0, 0]
    total_frequency_by_class = [0, 0, 0, 0]
    all_words = defaultdict(empty_class_list)
    examples_bags = {}

    for i in range(num_examples):
        word_list = data.iloc[i]["Description"].split()  # note that there is no punctuation in the training data, we also consider duplicates.
        examples_bags[i] = word_list

        class_index = class_indexes[data.iloc[i]["Class"]]

        numbers_of_classes[class_index] += 1

        for word in word_list:
            all_words[word][class_index] += 1
            total_frequency_by_class[class_index] += 1

    return numbers_of_classes, total_frequency_by_class, all_words, examples_bags

In [4]:
def track_frequencies_and_transform(data, num_examples, class_indexes, b):  # Uses transforming by document frequency

    numbers_of_classes = [0, 0, 0, 0]
    total_frequency_by_class = [0, 0, 0, 0]
    all_words = defaultdict(empty_class_list)
    words_document_occurances = Counter()
    examples_bags = {}

    for i in range(num_examples):

        word_list = data.iloc[i]["Description"].split()  # note that there is no punctuation in the training data, we also consider duplicates.
        class_index = class_indexes[data.iloc[i]["Class"]]
        examples_bags[i] = word_list

        word_set = list(set(word_list))
        words_document_occurances.update(word_set)  # this needs to be generated before we can calculate our frequency transformations

        numbers_of_classes[class_index] += 1


    for i in range(num_examples):
        example_bag = examples_bags[i]

        class_index = class_indexes[data.loc[i, "Class"]]

        words_freq_in_document = Counter(example_bag)

        for word in words_freq_in_document:
            transformed_freq = words_freq_in_document[word] * math.log(num_examples / words_document_occurances[word], b)

            all_words[word][class_index] += transformed_freq
            total_frequency_by_class[class_index] += transformed_freq


    return numbers_of_classes, total_frequency_by_class, all_words, examples_bags

In [5]:
def train(num_examples, project_classes, class_indexes, numbers_of_classes, total_frequency_by_class, all_words):  # calculates all probabilities

    # Prior probabilities
    priors = {project_classes[i]: numbers_of_classes[i] / num_examples for i in range(len(project_classes))}

    # Conditional probabilities
    laplace_top = 1
    laplace_bottom = len(all_words)

    conditionals = defaultdict(empty_class_list)
    for word in all_words:

        word_counts = all_words[word]

        for class_index in class_indexes.values():

            word_count = word_counts[class_index]
            class_length = total_frequency_by_class[class_index]

            conditionals[word][class_index] += (word_count + laplace_top) / (class_length + laplace_bottom)

    return priors, conditionals, laplace_top, laplace_bottom 
    # we pass the laplace values between functions so we don't need to keep recalculating them (and easier if we want to experiment with them)

In [6]:
def predict_example(example_bag, priors, conditionals, class_indexes, laplace_top, laplace_bottom):  # Using the naive bayesian algorithm

    likelyhoods = np.zeros(4)

    for project_class in class_indexes:

        prior = priors[project_class]

        conditionals_product = 1
        for word in example_bag:
            if word in conditionals:
                conditionals_product *= conditionals[word][class_indexes[project_class]]
                conditionals_product *= 500  # multiply by constant so the computer can actually store the final answer
            else:
                conditionals_product *= (laplace_top / laplace_bottom)  # laplace smoothing added for previously unseen words
            
        likelyhoods[class_indexes[project_class]] = prior * conditionals_product
        #print(prior * conditionals_product)
        

    prediction_index = np.argmax(likelyhoods)

    return list(class_indexes.keys())[prediction_index]
    

def predict_all(examples_bags, priors, conditionals, class_indexes, laplace_top, laplace_bottom):  # predicts all examples given dictionary of bags
    Y_list = []

    for example_bag in examples_bags.values():
        Y_list.append(predict_example(example_bag, priors, conditionals, class_indexes, laplace_top, laplace_bottom))

    return pd.DataFrame({"Class": Y_list})

In [7]:
def prepare_bags(data):
    num_examples = len(data)

    examples_bags = {}
    for i in range(num_examples):
        word_list = data.iloc[i]["Description"].split()  # note that there is no punctuation in the training data, we also consider duplicates.
        examples_bags[i] = word_list

    return examples_bags

In [8]:
def split_into_folds(data, k):
    data_size = len(data)
    fold_size = math.ceil(data_size / k)
    folds = {}

    for i in range(k):
        kth_fold = data.iloc[i*fold_size:(i+1)*fold_size]
        folds[i+1] = kth_fold

    return folds, fold_size


def accuracy(Y_true, Y_pred):
    total = len(Y_true)

    correct_predictions = 0
    for i in range(total):
        if Y_true[i] == Y_pred[i]:
            correct_predictions += 1
    
    return correct_predictions / total


def kcross_validation(data, k, project_classes, class_indexes, b, with_transform=True):
    folds, fold_size = split_into_folds(data, k)

    accuracies = []

    for i in range(k):
        training_folds = [folds[fold] for fold in folds if i+1 != fold]
        data_train = pd.concat(training_folds)
        training_size = len(data_train)
        data_validation = folds[i+1]
        
        if not with_transform:
            numbers_of_classes, total_frequency_by_class, all_words, examples_bags = track_frequencies(data_train, training_size, class_indexes)
        else:
            numbers_of_classes, total_frequency_by_class, all_words, examples_bags = track_frequencies_and_transform(data, training_size, class_indexes, b)

        priors, conditionals, laplace_top, laplace_bottom = train(training_size, project_classes, class_indexes, numbers_of_classes, total_frequency_by_class, all_words)
        validation_bags = prepare_bags(data_validation)

        validation_prediction = predict_all(validation_bags, priors, conditionals, class_indexes, laplace_top, laplace_bottom)

        acc = accuracy(list(data_validation["Class"]), list(validation_prediction["Class"]))
        accuracies.append(acc)
        
    return statistics.mean(accuracies)

In [9]:
def optimise_base(lower, upper, max_tries, epsilon_scalar, data_train, k, project_classes, class_indexes, in_logspace=True):  # modified binary search

    tries = 0
    while tries < max_tries:

        if in_logspace:  # if we have a massive range e.g. e~2.7 to 1,000,000 then this may be more effective
            lower_base = math.log(lower, 10)
            upper_base = math.log(upper, 10)
            mid = np.logspace(lower_base, upper_base, num = 3)[1]
            #print(str(np.logspace(lower_base, upper_base, num = 3)))
        else:
            lower = lower
            upper = upper
            mid = (lower + upper) / 2
            #print(str(lower) + " " + str(mid) + " " + str(upper))

        epsilon = epsilon_scalar * (upper - mid)  # scaling our epsilon      

        mid_acc = kcross_validation(data_train, k, project_classes, class_indexes, mid)
        mid_plus_e_acc = kcross_validation(data_train, k, project_classes, class_indexes, mid+epsilon)
        tries += 2

        if mid_acc < mid_plus_e_acc:  # accuracy is increasing after this point
            lower = mid + epsilon
        else:  # accuracy is decreasing after this point 
            upper = mid

    return mid, kcross_validation(data_train, k, project_classes, class_indexes, mid)

In [10]:
def predict_new_data(data_new, data_train, training_size, project_classes, class_indexes, b, with_transform=True):
    
    if not with_transform:
        numbers_of_classes, total_frequency_by_class, all_words, examples_bags = track_frequencies(data_train, training_size, class_indexes)
    else:
        numbers_of_classes, total_frequency_by_class, all_words, examples_bags = track_frequencies_and_transform(data_train, training_size, class_indexes, b)
    
    priors, conditionals, laplace_top, laplace_bottom = train(training_size, project_classes, class_indexes, numbers_of_classes, total_frequency_by_class, all_words)
    data_new_bags = prepare_bags(data_new)

    final_prediction = predict_all(data_new_bags, priors, conditionals, class_indexes, laplace_top, laplace_bottom)

    return final_prediction

In [11]:
# Loading data

file_path_train = "./data/train.csv"
data_train = pd.read_csv(file_path_train)
project_classes = ["A", "S", "G", "W"]
class_indexes = {proj_class: index for index, proj_class in enumerate(project_classes)}
training_size = len(data_train)

In [12]:
# Run this cell to optimise the base hyper-parameter for the method of Transforming by Document Frequency

k = 10  # number of folds in k-fold validation
best_base = optimise_base(math.e, 1000000, 10, 0.1, data_train, k, project_classes, class_indexes, True)
print(best_base)  # prints the best base found and its validation accuracy

(np.float64(158405.4522032182), 0.9811363636363636)


In [13]:
# Run this cell to find validation accuracy for either basic naive bayes or naive bayes with transformation

with_tranformation = False  # basic naive bayes or bayes with transformation
k = 10  # number of folds in k-fold validation
b = math.e  # base hyper-parameter for the method of Transforming by Document Frequency
validation_score = kcross_validation(data_train, k, project_classes, class_indexes, b, with_tranformation)
print(validation_score)

0.9604545454545454


In [ ]:
# Run this cell to predict a set of new examples

file_path_test = "./data/test.csv"
data_test = pd.read_csv(file_path_test)

b = 158405  # found using our hyper-parameter optimisation

use_extended_version_with_transform = True
final_prediction = predict_new_data(data_test, data_train, training_size, project_classes, class_indexes, b, use_extended_version_with_transform)

final_prediction.index = range(1, len(final_prediction) + 1)
final_prediction.index.name = 'Id'
final_prediction.to_csv('final_prediction.csv', index=True)

## Description

### Preprocessing & Standard Naive Bayesian Learner
We first sort each example into a bag of words. Each "bag" is a list and is stored in a dictionary with the index of the example as the key. We allow for duplicate words, this is consistent with the examples shown in lectures. During this process we also keep track of: the number of occurances of each word in a class; the total number of words per class; the quantities of each class. This should give us all the ingredients we need for our standard naive Bayesian (SNB) classifier.

Earlier versions used an attribute matrix instead of the bags of words method, but it was harder to work with while building the classifier and offered no noticable computational benefits. Bags-of-words is easier to implement, easier to work with (as one can simply iterate through the examples and the words within the examples), and not computationally expensive. Generating these bags at this stage will also make future improvements (e.g. adding n-grams) easier to implement.

The various frequency counts are the most important part of this preprocessing section for the training phase. Total word counts and class counts are stored in lists of length 4, with indices corresponding to classes (0 for "A", 1 for "S", etc). Numbers of occurances of each word per class are stored in a dictionary where each word maps to a list of length 4 (indexed as previously described). This dictionary also serves as the vocabulary.

All the above is done in our track_frequencies() function, we now enter the train() function which calculates all the probabilities from the results of the previous function. We first calculate our prior probabilities (class count divided by number of examples) and store these in a dictionary. We then calculate our conditional probabilities (with laplace smoothing where m=|X| is the number of unique words so that words that don't appear in a class don't render that value useless). These are stored in a dictionary with each word mapped to a list of its conditional probability with each class.

Examples can now be predicted (with predict_all() and predict_example() functions) using the SNB formula (with laplace smoothing for unseen words). Note that each time we multiply by a conditional probability, we also multiply by a constant in order to keep our result in a reasonable range for our computer to store (experimentation found c=500 to work). For a given example we multiply each likelyhood by the same amount, so this operation doesn't change the final prediction.


### Evaluation: Validation

We implement a standard k-cross validation method. We split the data into k folds and allow for the final fold to be smaller than the rest in the case that k doesn't divide our number of examples evenly. The training folds are then concatenated and trained (on either our standard model or improved model), validated against the validation fold (using the Accuracy metric), and stored in a list. This is repeated for each fold and the final accuracy score is returned as the mean of each of the validation folds' accuracy. This method gives us a fairly accurate estimate of the test error we can expect (especially with high k), but is somewhat computationally expensive.


### Model Improvement: Transforming by Document Frequency

Section 4.2 of the paper "Tackling the Poor Assumptions of Naive Bayes Text Classifiers" (J. Rennie, L. Shih, J. Teevan, and D. Karger, available at https://people.csail.mit.edu/jrennie/papers/icml03-nb.pdf) describes how word frequency in a document, $f_i$, can be scaled as $f_i^\prime = f_i \log \frac{\sum_j 1}{\sum_j \delta_{ij}}$ where $\delta_{ij}$ is 1 if word $i$ occurs in document $j$, 0 otherwise. This downweights words that appear in many documents and upweight words that appear in few. The base of the logarithm is a hyper-parameter, a higher base will penalise commonly appearing words more. Intuitively, this seems like it would improve performance. Words such as "we", "that", and "to" appear often in training data and don't seem to correlate to any particular class, but if such words happen to occur more in a particular class in our training data this would create bias and hurt performance. Less common words are more likely to be domain-specific and be effective at predicting that class.

Implementing this improvement requires us to change the way we track our various frequencies. We want to first track the number of documents a given word occurs in. Then, instead of recording each frequency as normal, we instead first perform the transformation and take this as our word frequency. We then return everything in the same format as our standard preprocessing from before. This allows us to use the same prediction functions, but now they are predicting based on our transformed data.

We can now explore how to optimise our logarithm base, b, hyper-parameter. Some quick expermentation found that b=e gives $\approx 96\%$ validation score, b=100,000 gives $\approx 98\%$, and b=1,000,000 gives $\approx 97\%$. If we make the assumption that larger bases will increase our accuracy to a maximum, and then accuracy scores will decrease, we can make an algorithm that can find this maximum value. We can modify a basic binary search algorithm (with e.g. e as a minimum and 1,000,000 as a maximum) to check if accuracy is increasing or decreasing at a mid point (by validating at this point and at some small $\varepsilon$ distance nearby) and change our search range accordingly. We can run this in a linear space or logspace (which may be better due to the scale of the numbers we are dealing with). As validation scores are expensive, we will only do this a small number of times. Testing a small number of hyper-parameter values will also minimise our risk of optimisation bias. Running this in logspace using 10-fold cross validation and using 10 "guesses" we get an optimised b of 158405 with validation score $\approx 98.11\%$, which is near to the 100,000 guess from earlier.

### Validation Results

We use 10-fold cross validation so that computations complete in reasonable time. Our SNB model achieves a validation score of $\approx 96.0\%$ and our improved model with optimised hyper-parameter achieves a validation score of $\approx 98.1\%$ which is a notable improvement as expected. As previously mentioned, our hyper-parameter optimisation relies on the assumption that we have a single global maximum and this may not be true. It is possible algorithm that is more robust to local maxima may find a better base for our logarithm, but this method with our assumption still manages to improve performance.